In [1]:
import os
import glob
import shutil
from tqdm.notebook import tqdm
from librosa.display import specshow
from IPython.display import Audio
from joblib import Parallel, delayed


import sys
sys.path.append('/home/ci411/pcen-t-varying/')

from pcen_t.utils import *

In [2]:
audio_path = "/beegfs/ci411/pcen/reverb_pitch_URBAN-SED/audio"
pcen_path = "/beegfs/ci411/pcen/features_807/pcen"
mel_path = "/beegfs/ci411/pcen/features_807/mel"

feature_suffixes = ["dry", "alley", "bedroom", "tunnel", "sim_short", "sim_medium", "sim_long"]
subdirs = ["train", "validate", "test"]

'''
#spot check audio/features
test_file = 'soundscape_validate_bimodal662'
semitone = '0'
test_audio = os.path.join(audio_path, 'validate', test_file + '_{}.wav'.format(semitone))
test_pcen = os.path.join(pcen_path,'URBAN-SED_reverb', 'validate', test_file +'_{}.h5'.format(semitone))
pcen = load_h5(test_pcen)['PCEN/mag'][0,:,:,4]
test_mel = os.path.join(mel_path, 'URBAN-SED_reverb', 'validate', test_file +'_{}.h5'.format(semitone))
mel = load_h5(test_mel)['Mel/mag'][0,:,:,0].T
pcen = load_h5(test_pcen)['PCEN/mag'][0,:,:,2].T

Audio(test_audio)

print(mel.shape)
specshow(mel)

print(pcen.shape)
specshow(pcen)
'''

"\n#spot check audio/features\ntest_file = 'soundscape_validate_bimodal662'\nsemitone = '0'\ntest_audio = os.path.join(audio_path, 'validate', test_file + '_{}.wav'.format(semitone))\ntest_pcen = os.path.join(pcen_path,'URBAN-SED_reverb', 'validate', test_file +'_{}.h5'.format(semitone))\npcen = load_h5(test_pcen)['PCEN/mag'][0,:,:,4]\ntest_mel = os.path.join(mel_path, 'URBAN-SED_reverb', 'validate', test_file +'_{}.h5'.format(semitone))\nmel = load_h5(test_mel)['Mel/mag'][0,:,:,0].T\npcen = load_h5(test_pcen)['PCEN/mag'][0,:,:,2].T\n\nAudio(test_audio)\n\nprint(mel.shape)\nspecshow(mel)\n\nprint(pcen.shape)\nspecshow(pcen)\n"

In [3]:
#make subdirectories for class-specific folders
'''
for feature in feature_suffixes:
    os.mkdir(os.path.join(pcen_path, "URBAN-SED_"+feature))
    os.mkdir(os.path.join(mel_path, "URBAN-SED_"+feature))
    for subdir in subdirs:
        os.mkdir(os.path.join(pcen_path, "URBAN-SED_"+feature, subdir))
        os.mkdir(os.path.join(mel_path, "URBAN-SED_"+feature, subdir))
'''

'\nfor feature in feature_suffixes:\n    os.mkdir(os.path.join(pcen_path, "URBAN-SED_"+feature))\n    os.mkdir(os.path.join(mel_path, "URBAN-SED_"+feature))\n    for subdir in subdirs:\n        os.mkdir(os.path.join(pcen_path, "URBAN-SED_"+feature, subdir))\n        os.mkdir(os.path.join(mel_path, "URBAN-SED_"+feature, subdir))\n'

In [5]:

#move pcen features to class-specific folders
for feature in tqdm(feature_suffixes):
    for subdir in tqdm(subdirs):
        matching = os.path.join(pcen_path, "URBAN-SED_reverb", subdir, "*_"+feature+'*.h5')
        dest_dir = os.path.join(pcen_path, "URBAN-SED_" + feature, subdir)
        files = glob.glob(matching)
        #Parallel(n_jobs=4)(delayed(shutil.move)(file, dest_dir) for file in files)

#move mel features to class-specific folders
for feature in tqdm(feature_suffixes):
    for subdir in tqdm(subdirs):
        matching = os.path.join(mel_path, 'URBAN-SED_reverb', subdir, "*_"+feature+'*.h5')
        dest_dir = os.path.join(mel_path, "URBAN-SED_" + feature, subdir)
        files = glob.glob(matching)
        #Parallel(n_jobs=4)(delayed(shutil.move)(file, dest_dir) for file in files)


#move remaining dry samples to dry folder
for subdir in tqdm(subdirs):
    matching = os.path.join(pcen_path, "URBAN-SED_reverb", subdir, "*.h5")
    dest_dir = os.path.join(pcen_path, "URBAN-SED_dry", subdir)
    files = glob.glob(matching)
    #Parallel(n_jobs=4)(delayed(shutil.move)(file, dest_dir) for file in files)
                            
    matching = os.path.join(mel_path, "URBAN-SED_reverb", subdir, "*.h5")
    dest_dir = os.path.join(mel_path, "URBAN-SED_dry", subdir)
    files = glob.glob(matching)
    #Parallel(n_jobs=4)(delayed(shutil.move)(file, dest_dir) for file in files)


In [10]:
pcen_dest_path = "/beegfs/ci411/pcen/features_807/unpitched/pcen"
mel_dest_path = "/beegfs/ci411/pcen/features_807/unpitched/mel"

#make directory structure in unpitched folders
'''
for feature in feature_suffixes:
    os.mkdir(os.path.join(pcen_dest_path, "URBAN-SED_"+feature))
    os.mkdir(os.path.join(mel_dest_path, "URBAN-SED_"+feature))
    for subdir in subdirs:
        os.mkdir(os.path.join(pcen_dest_path, "URBAN-SED_"+feature, subdir))
        os.mkdir(os.path.join(mel_dest_path, "URBAN-SED_"+feature, subdir))
'''

#move un-shifted audio to a separate folder
features_tqdm = tqdm(feature_suffixes[5:])
for feature in features_tqdm:
    features_tqdm.set_description(feature)
    subdirs_tqdm = tqdm(subdirs)
    for subdir in subdirs_tqdm:
        subdirs_tqdm.set_description("PCEN")
        matching = os.path.join(pcen_path, "URBAN-SED_" + feature, subdir, "*_0.h5")
        dest_dir = os.path.join(pcen_dest_path, "URBAN-SED_" + feature, subdir)
        files = glob.glob(matching)
        Parallel(n_jobs=4)(delayed(shutil.copy)(file, dest_dir) for file in files)

        subdirs_tqdm.set_description("Mel")
        matching = os.path.join(mel_path, "URBAN-SED_" + feature, subdir, "*_0.h5")
        dest_dir = os.path.join(mel_dest_path, "URBAN-SED_" + feature, subdir)
        files = glob.glob(matching)
        Parallel(n_jobs=4)(delayed(shutil.copy)(file, dest_dir) for file in files)